# Project 3: Linear regression with House Prices


## Introduction:
- In this notebook we will expereiment with a dataset that is provided from kaggle about predicting the price of a home with 79 features of homes in  Ames Iowa
- The goal is to use linear regression and other techniques to predict the price of a home with the parameters a person wants
- The Link of the kaggle dataset is here : https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview 


## Features
- there are 79 features in this dataset.



## prequiestist

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk

In [4]:
# Load data from datasets
trainingDataset= pd.read_csv("train.csv")

In [8]:
# feature lokking he
trainingDataset.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

In [9]:
# print  the result if any column has null values.
print(f"\nNull values %:\n {trainingDataset.apply(lambda x: sum(x.isnull()) / len(trainingDataset))}")
 


Null values %:
 Id               0.000000
MSSubClass       0.000000
MSZoning         0.000000
LotFrontage      0.177397
LotArea          0.000000
                   ...   
MoSold           0.000000
YrSold           0.000000
SaleType         0.000000
SaleCondition    0.000000
SalePrice        0.000000
Length: 81, dtype: float64
